In [2]:
import requests
import bs4
import nltk
# Load, explore, process and plot data
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as pk

#Text Preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')
nltk.download('omw-1.4')

In [ ]:
def get_blog_url(soup):
    div_ = soup.find_all('div', attrs = {'class':'FL PR20'})
    url_list = []
    for title in div_:
        href = title.find('a')['href']
        url_list.append("https://www.moneycontrol.com/"+href)
    return url_list

In [ ]:

def get_blog_content(url):
    request = requests.get(url)
    print('Received URL',url)
    soup = bs4.BeautifulSoup(request.text,'html.parser')
    all_script = soup.find_all('script',attrs = {'type':'application/ld+json'})
    raw_article_str = all_script[2].get_text().replace('\r\n',' ')
    parts = re.split(r"""("[^"]*"|'[^']*')""", raw_article_str)
    parts[::2] = map(lambda s: "".join(s.split()), parts[::2])
    article_str = "".join(parts)
    article_str = article_str[1:]
    article_str = article_str[:-1]
    article_dict = json.loads(article_str, strict=False)
    all_tags = soup.find_all('div',attrs={'class':'tags_first_line'})
    list_all_tags=[]
    for i in all_tags:
        list_all_tags.append(i.get_text())
    tags = list_all_tags[0].replace('Tags','')
    tags = tags.replace('\n','')
    tags = tags.split('#')
    tags = tags[1:]
    tags = ', '.join([str(elem).strip() for elem in tags])
    article_dict['tags'] = tags
    return article_dict

In [ ]:
def get_page_no(url, sc_id, page_no, next, year):
    request = requests.get(url)
    soup = BeautifulSoup(request.text,'html.parser')

    
    all_page_no = soup.find_all('div', attrs = {'class': 'pages MR10 MT15'})
    page_list  = [i.text for i in all_page_no[0].find_all('a')]


    if any(map(str.isdigit,page_list[-1])):
        return int(page_list[-1]), next
    else:
        next = next + 1
        page_no = int(page_list[-2])
        url = "https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc_id="+sc_id+"&scat=&pageno="+str(page_no)+"&next="+str(next)+"&durationType=Y&Year="+str(year)+"&duration=1&news_type="
    return get_page_no(url, sc_id, page_no, next, year)
    



In [ ]:
def save_company_data(url_ = "https://www.moneycontrol.com/stocks/company_info/stock_news.php?", sc_id=[], pageno=1, next=0, years=[]):
    for company in sc_id:
        df = pd.DataFrame(columns = ['company', 'datePublished', 'author', 'headline',
                                     'description', 'articleBosy','tags','url'])
        for year in years:
            print('Year: ', year)
            print('Page_No:', pageno)
            print('Next: ', next)

            url = url_ + "sc_id="+company+"&scat=&pageno="+str(pageno)+"&next="+str(next)+"&durationType=Y&Year="+str(year)+"&duration=1&news_type="
            print('url: ', url)

            max_page_no, max_next = get_page_no(url, company, pageno, next, year)
            max_next = max_next + 1
            print('Total Page:',max_page_no)
            for i in range(max_next):
                for j in range((i*10)+1, (i*10)+11):
                    if j <= max_page_no:
                        url_list = []
                        url = url_ + "sc_id="+company+"&scat=&pageno="+str(j)+"&next="+str(i)+"&durationType=Y&Year="+str(year)+"&duration=1&news_type="
                        print(url)
                        request = requests.get(url)
                        soup = BeautifulSoup(request.text, 'html.parser')
                        
                        url_list = get_blog_url(soup)
                        
                        frame1 = []

                        
                        for url in url_list:
                            try:
                                #print('Blog URL:',url)
                                article_dict = get_blog_content(url)

                                
                                print(company)
                                print(article_dict['datePublished'])
                                print(article_dict['author'])
                                print(article_dict['headline'])
                                print(article_dict['description'])
                                print(article_dict['articleBody'])
                                print(article_dict['tags'])
                                print(article_dict['url'])
                                print('--------------------------------------------------------')

                                article_lst = [[company,
                                                 article_dict['datePublished'],
                                                 article_dict['author'],
                                                 article_dict['headline'],
                                                 article_dict['description'],
                                                 article_dict['articleBody'],
                                                 article_dict['tags'],
                                                 url]]
                               
       
                                df = pd.concat([df,pd.DataFrame(article_lst, columns=['company','datePublished','author','headline',
                                                                                   'description','articleBody','tags','url'])],axis=0, ignore_index = True)
                               

                            except:
                                
                                article_lst = [[company, 'error','error','error','error','error','error',url]]
                                
     
                                df = pd.concat([df,pd.DataFrame(article_lst, columns=['company','datePublished','author','headline',
                                                                                   'description','articleBody','tags','url'])],axis=0, ignore_index = True)
                               
                                continue
                           
        df.to_csv(company+'.csv')

In [ ]:
save_company_data(sc_id=['RI'], years = [2008,2007])

In [3]:
df = pd.read_csv('News_Data/RI(2011_2023).csv')

In [4]:
df.head()
df.columns

Index(['S. No.', 'company', 'datePublished', 'author', 'headline',
       'description', 'tags', 'url', 'articleBody'],
      dtype='object')

In [5]:
df.columns
df = df.drop(['S. No.'], axis=1)

In [6]:
article_sentiments = pd.DataFrame({
                        'neg':[],
                        'neu':[],
                        'pos':[],
                        'compound':[]
})

In [8]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

for text in df.articleBody:
    sid = SentimentIntensityAnalyzer()
    polarity = sid.polarity_scores(str(text))    
    tmpdic = {}    
    tmpdic.update(polarity)
    article_sentiments= pd.concat([article_sentiments,pd.DataFrame([tmpdic])], ignore_index=True)    
    
    


In [9]:
article_sentiments.to_csv("News_Data/article_sentiments_RI_2011-2023.csv", sep=',', encoding='utf-8', header=True)